**Общая информация**

**Срок сдачи:** 21 октября 2020, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [74]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from scipy.spatial.distance import cdist
from sklearn.neighbors import KDTree

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [158]:

class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
    
    def fit(self, X, y):
        if(self.algorithm == 'brute'):
            self.X = X
            self.y = y
        elif(self.algorithm == 'kd_tree'):
            self.tree = KDTree(X)
            self.y = y
    
    def get_nearest_neighbors(self, distances):
        indexes = np.argpartition(-distances, -self.n_neighbors, axis=1)[:, -self.n_neighbors:]
        array_top = np.take_along_axis(distances, indexes, axis=1)
        sort_index = np.argsort(array_top, axis=1)
        return np.take_along_axis(indexes, sort_index, axis=1)
    
    def predict(self, X):
        if(self.algorithm == 'brute'):
            distances = cdist(X, self.X)
            indexes = self.get_nearest_neighbors(distances)
        else:
            indexes = self.tree.query(X, self.n_neighbors)[1]
        nearest_neighbors = self.y[indexes]
        return np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), 1, nearest_neighbors)
    

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [159]:
iris = datasets.load_iris()

In [160]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [161]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [162]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [163]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [164]:
%time clf.fit(X_train, y_train)

CPU times: user 1.11 ms, sys: 0 ns, total: 1.11 ms
Wall time: 993 µs


KNeighborsClassifier(algorithm='brute', n_neighbors=2)

In [165]:
%time my_clf.fit(X_train, y_train)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 8.58 µs


In [166]:
%time clf.predict(X_test)

CPU times: user 2.45 ms, sys: 0 ns, total: 2.45 ms
Wall time: 2.17 ms


array([1, 1, 2, 0, 1, 0, 0, 2, 2, 0, 2, 0, 1, 2, 2])

In [167]:
%time my_clf.predict(X_test)

CPU times: user 1.38 ms, sys: 0 ns, total: 1.38 ms
Wall time: 1.03 ms


array([1, 1, 2, 0, 1, 0, 0, 2, 2, 0, 2, 0, 1, 2, 2])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [168]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [169]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [170]:
%time clf.fit(X_train, y_train)

CPU times: user 1.42 ms, sys: 126 µs, total: 1.54 ms
Wall time: 1.4 ms


KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)

In [171]:
%time my_clf.fit(X_train, y_train)

CPU times: user 152 µs, sys: 16 µs, total: 168 µs
Wall time: 160 µs


In [172]:
%time clf.predict(X_test)

CPU times: user 2.34 ms, sys: 117 µs, total: 2.46 ms
Wall time: 2 ms


array([2, 2, 1, 0, 2, 0, 0, 0, 1, 1, 1, 0, 2, 1, 2])

In [173]:
%time my_clf.predict(X_test)

CPU times: user 0 ns, sys: 1.81 ms, total: 1.81 ms
Wall time: 1.82 ms


array([2, 2, 1, 0, 2, 0, 0, 0, 1, 1, 1, 0, 2, 1, 2])

In [174]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [175]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [176]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.